In [1]:
import numpy as np
import matplotlib.pylab as plt
from pylab import *
import sys
import pandas as pd

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
##########################
# define a few constants #		##fft python random phase
##########################

e0 = np.loadtxt('/content/drive/MyDrive/models/837nm.dat')		#load the reference pulse, pump @836 nm
e0 = e0.T
e1 = np.sqrt(e0[1])
e1_ph = e0[2]
e = np.array(e1)*np.exp(-1j*(np.array(e1_ph)))		#reforming E-field

original_trace = np.loadtxt('/content/drive/MyDrive/models/trace/trace1.frg')		#load FROG trace from experiment

N = len(e0[0])		#timesteps
N_max = np.max(e0[0])
N_min = np.min(e0[0])

t = e0[0]-N_min
T0 = N_max-N_min	#total time duration
dt = T0/(N-1)
df = 1.0/T0		#frequency steps
TD = N			#time delay for FROG-trace generation
freq = np.arange(0,1/dt+df,df)*1000

In [4]:
original_trace.shape

(128, 128)

In [5]:
## for generation of Gaussian pulses
def gaussian_pulse(T,f,A,phase,tau):
	w0 = f*2*np.pi		#central angular frequency of pulse
	temp = -1j*w0*(t-tau) + 1j*phase
	E = A*np.sqrt(np.exp(-4.0*np.log(2.0)/np.log(np.e)*((t-tau)/T)**2))*np.exp(-temp)
	return E

In [6]:
## calculate cross correlation of two pulses
def xcorr(e0,e,td):		#shift one pulse by time delay of 'td'
	E_delay = e*1.0
	if td<=0:
		E_delay[0:N+td] = e0[-td:]
		E_delay[N+td:] = [0]*(-td)
	else:
		E_delay[0:td] = [0]*(td)
		E_delay[td:] = e0[0:N-td]	
	return  e*E_delay

In [7]:
## calculate the differert of trace A and trace B
def array_diff(A,B):
	A = A/np.max(A)
	B = B/np.max(B)
	return np.sum(np.square(A-B))/np.sum(np.square(A))

In [8]:
def FROG_generator(e,E,TD):	
	FROG = np.zeros((TD,N),dtype=float)
	for td in range(-TD//2,TD//2):
		E_corre = xcorr(e,E,td)
		temp0 = np.fft.fftshift(np.fft.fft(E_corre))
		FROG[(td+TD//2)-1] = abs(temp0)**2	
	return FROG.T

In [9]:
## plot out FROG trace
def FROG_trace_plot(FROG):
	fig,ax = plt.subplots()
	plt.imshow(FROG)
	plt.title("FROG trace")
	plt.grid(True)
	plt.show()

In [10]:
## calculate FROG traces with different time delays
def FROGs(T1,T2,A1,a2,a3,f0,f1,tau1):
    tau2 = tau1-700
    phase1 = a2*(t-tau1)**2+a3*(t-tau1)**3
    phase2 = a2*(t-tau2)**2+a3*(t-tau2)**3 
    E1 = gaussian_pulse(T1,f0,A1,phase1,tau1)
    E2 = gaussian_pulse(T2,f1,A2,phase2,tau2)
    E = E1+E2
    FROG = FROG_generator(e,E,TD)
    delta = array_diff(FROG, original_trace)	
    return FROG, delta

In [11]:
######################
# initial parameters #
######################

T1 = np.arange(295,695)                  #Gaussian pulse duration in fs, FW
T2 = np.arange(300,1000)
#T2 = 287.7733748374851
A1 = np.arange(0.6,1.5,0.05)
#A1 = 0.7524039393439459                #Amplitude of Raman pulse
#A2 = np.arange(0.6,1.5,0.05)
A2 = 1

#a2 = 2.50*10**-6
a2 = np.arange(-4e-6, 1e-6, 1e-8)
a2 = np.delete(a2, np.where(np.abs(a2) < 1e-10))#second order phase factor
#a3 = 3.0*10**-9
a3 = np.arange(-1.45e-9, 1.5e-9, 1e-11)		#third order phase factor
a3 = np.delete(a3, np.where(np.abs(a3) < 1e-14))

F0 = np.arange(-0.005, 0.005, 0.0005)
F0 = np.delete(F0, np.where(np.abs(F0) < 1e-10))    #Raman center-wavelength
#f0 = 0.0014144712047722094
F1 = np.arange(-0.0005, 0.0005, 0.00005)
F1 = np.delete(F1, np.where(np.abs(F1) < 1e-10))    #Red-shifted center-wavelength
#f1 = 0.0002521943119991874
			
i0 = 333.0		#unit: fs
Tau1 = np.arange(-1000, 5000, 50)	#Raman pulse peak time
#tau1 = 3325.2480418676446
#tau2 = tau1-700		#Red-shifted pulse peak time

In [12]:
values = pd.DataFrame(columns = ['T1', 'T2', 'A1', 'a2', 'a3', 'F0', 'F1', 'Tau1'])
values

,T1,T2,A1,a2,a3,F0,F1,Tau1


In [13]:
###########################
# FROG trace main program #
###########################
FROGS = []
values = []
for i in range(35000):
    t1 = np.random.choice(T1)
    t2 = np.random.choice(T2)
    ai1 = np.random.choice(A1)
    #ai2 = np.random.choice(A2)
    f0 = np.random.choice(F0)
    f1 = np.random.choice(F1)
    aa2 = np.random.choice(a2)
    aa3 = np.random.choice(a3)
    tau1 = np.random.choice(Tau1)
    (FROG, delta) = FROGs(t1,t2,ai1,aa2,aa3,f0,f1,tau1)
    print(i, delta)
    values.append([t1, t2, ai1, aa2, aa3, f0, f1, tau1])
    FROGS.append(FROG)

Streaming output truncated to the last 5000 lines.
30000 1.3294660931988918
30001 2.1151254965808026
30002 1.2177499765865387
30003 2.674841737843439
30004 2.5464124946659066
30005 1.650416075350009
30006 1.676124348687367
30007 2.6018599824019284
30008 1.6265215928259293
30009 1.700507566219707
30010 1.6120268119700671
30011 1.6579771705362738
30012 2.548641765916112
30013 0.5176804382209949
30014 0.7367192504764142
30015 1.014969048829903
30016 2.182591501825753
30017 2.1266270883065417
30018 2.5226605246486913
30019 1.7368808997374123
30020 1.7196718891179887
30021 0.9401751041772533
30022 1.8041893932352562
30023 1.4282518430777864
30024 1.9577420839468436
30025 2.106939856815843
30026 1.9841235704630864
30027 2.13186997395073
30028 1.3098933772653718
30029 2.057484379796764
30030 0.8839500675664799
30031 1.9181134144586698
30032 2.3233161926840387
30033 2.5621007393530033
30034 2.4466704526341116
30035 2.104183914921335
30036 2.481490737280541
30037 2.4767864488135762
30038 1.7075

In [14]:
FROGS = np.array(FROGS)
#FROGS = FROGS.reshape(1000,-1)

In [15]:
values = np.array(values)
values.shape

(35000, 8)

In [16]:
np.save("/content/drive/MyDrive/models/FROG_8p_500K", FROGS)

In [17]:
np.save("/content/drive/MyDrive/models/labels_ann_8p_500K", values)